In [4]:
# Importation des bibliothèques
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import mplfinance as mpf
import numpy as np
import os

class Backtester:
    # Initialisation de la classe
    def __init__(self, strategy):
        self.strategy = strategy
        self.positions = []
        
    # Compute : récupération, stockage des données et application de la stratégie
    def compute(self, symbols, start_date, end_date):
        # Déclaration des bibliothèque
        self.df_market={}  # Data frame pour les indices de marché
        self.symbols={} # Data frame pour les symboles
        self.df={} # Data frame pour les données des titres
        # Récupération et stockage local des données : Indices de marché
        markets = ['^GSPC','^FCHI','^GSPC', 'ACWI']  # Symbole pour l'indice S&P 500, euro, CAC40, World
        for market in markets:
            index_name = f"{market, start_date, end_date}_data.csv"
            if os.path.exists(index_name):
                market_data = pd.read_csv(index_name, index_col=0)
            else:
                market_data =yf.download(market, start=start_date, end=end_date)
                market_data.to_csv(index_name)
            self.df_market[market]=pd.DataFrame(market_data)
            # Déterminer les rendements positives et négatives de l'indice
            self.df_market[market]['positive'] = np.where(self.df_market[market]['Close'].pct_change() > 0, self.df_market[market]['Close'].pct_change(), np.nan)
            self.df_market[market]['negative'] = np.where(self.df_market[market]['Close'].pct_change() < 0, self.df_market[market]['Close'].pct_change(), np.nan)
        for symbol in symbols:
            # Récupération des symboles
            self.symbols[symbol] = symbol
            # Récupération et stockage local des données des titres
            file_name = f"{symbol, start_date, end_date}_data.csv"
            if os.path.exists(file_name):
                data = pd.read_csv(file_name, index_col=0)
            else:
                data = yf.download(symbol, start=start_date, end=end_date)
                data.to_csv(file_name)
            self.df[symbol] = pd.DataFrame(data)
            # Calcul des résultats pour la stratégie
            position = self.strategy(self.df[symbol])
            self.df[symbol]['position'] = position
        return 
       
    # Summary : indicateurs de performance de la stratégie
    def summary(self):
        # Déclaration des dictionnaires
        average_strategy_return = {}
        kurtosis = {}
        cumulative_strategy_return = {}
        volatility = {}
        annual_volatility = {}
        beta = {}
        beta_haussier={}
        beta_baissier={}
        max_drawdown = {}
        skewness = {}

        # Rendements
        for symbol, position in self.positions:
            #Calcul des rendements du titres 
            self.df[symbol]['asset_returns']=self.df[symbol]['Close'].pct_change()
            #Calcul des rendements de la stratégie 
            self.df[symbol]['strategy_return']=self.df[symbol]['asset_returns']*self.df[symbol]['position'].shift(1)
            #print(self.df)

        for symbol in self.symbols:
            #Calcul des rendements cumulatifs 
            cumulative_strategy_return[symbol]=(1+self.df[symbol]['strategy_return'].dropna()).cumprod()-1
            self.df[symbol]['cumulative_strategy_return']=cumulative_strategy_return[symbol]
            print(cumulative_strategy_return[symbol].iloc[-1])
            #print(self.df)
            #print(cum_str_re[symbol])

            #Rendement moyen annuel 
            r_m=self.df[symbol]['strategy_return'].mean()*252
            #print(r_m)
            #print(symbol)

            #Rendement annuel
            self.df[symbol]['annual_return']=(1+self.df[symbol]['strategy_return']).cumprod() ** (252/len(self.df[symbol]['strategy_return']))-1
            annual_strategy_return=(1+cumulative_strategy_return[symbol].iloc[-1]) ** (252/len(self.df[symbol]['strategy_return']))-1
            #print(len(self.df[symbol]['strategy_return']))
            #print(annual_strategy_return)


            # Volatilité
            # Volatilité quotidienne 
            volatility[symbol] =np.std(self.df[symbol]['strategy_return'])
       
            # Volatilité annuelle 
            annual_volatility[symbol] = volatility[symbol]*np.sqrt(252)

            # Bêta
            for market in markets:
                beta[symbol][market]=(np.cov(self.df_market[market]['Close'].pct_change().dropna(),self.df['strategy_return'].dropna()))/(np.var(self.df_market[market]['Close'].pct_change().dropna()))
                # Bêta haussier
                beta_haussier[symbol][market]= (np.cov(self.df_market[market]['positive'].dropna(),self.df['strategy_return'].dropna()))/(np.var(self.df_market[market]['positive'].dropna()))
                # Bêta baissier
                beta_baissier[symbol][market]= (np.cov(self.df_market[market]['negative'].dropna(),self.df['strategy_return']))/(np.var(self.df_market[market]['negative'].dropna()))

            # Drawdowns 
            self.df[symbol]['drawdowns'] = self.df[symbol]['strategy_return'] - self.df[symbol]['strategy_return'].cummax()
            max_drawdown[symbol] = self.df[symbol]['drawdowns'].min()

            # Kurtosis
            kurtosis[symbol] = (self.df[symbol]['strategy_return'].dropna()).kurtosis()
            print(kurtosis[symbol])

            # Skewness
            skewness[symbol] = self.df[symbol]['strategy_return'].dropna().skew()
       
           # Ratio de Sharpe
           # Downside risk (DSR)
           # Ratio de sortino
        return
      
    # Plot : Visualisation des résultats
    def plot(self):
        # Graphique des performances
        return

# Exemple de stratégie
def simple_strategy(data):
    # Exemple de stratégie simple : achat lorsque la moyenne mobile sur 50 jours est supérieure à la moyenne mobile sur 200 jours, et vente dans le cas contraire
    data['50_MA'] = data['Close'].rolling(window=50).mean()
    data['200_MA'] = data['Close'].rolling(window=200).mean()
    data['Position'] = 0
    data.loc[data['50_MA'] > data['200_MA'], 'Position'] = 1  # Achat
    data.loc[data['50_MA'] < data['200_MA'], 'Position'] = -1  # Vente
    return data['Position']

# Exemple d'utilisation du backtester avec la stratégie simple
Backteste = Backtester(strategy=simple_strategy)
Backteste.compute(["MSFT", "AAPL"], "2021-01-01","2023-12-31")
print(Backteste.summary())
Backteste.plot()


KeyError: 'strategy_return'